**Ex1 (OP1)**

In [ ]:
!pip install -q pyomo
from pyomo.environ import * 
import numpy as np
import pandas as pd
!apt-get install -y -qq coinor-cbc

In [3]:
coef = np.loadtxt('lab6_practice_coef.txt', delimiter=',')

model_op1 = ConcreteModel()

N = coef.shape[1]-1
M = coef.shape[0]-1

obj_coef = coef[0,:-1]

constr_coef = coef[1:,:-1]

constr_rhs = coef[1:,-1]

col_indices_x = np.arange(int(N/2))
model_op1.x = Var(col_indices_x)

col_indices_u = np.arange(int(N/2))
model_op1.u = Var(col_indices_u, domain=NonNegativeReals)

row_indices = np.arange(M)

model_op1.constraints = ConstraintList()

for i in row_indices:
  model_op1.constraints.add(summation(constr_coef[i][0:int(N/2)],model_op1.x) + summation(constr_coef[i][int(N/2):N],model_op1.u)  >= constr_rhs[i])

model_op1.objective = Objective(expr = summation(obj_coef[0:int(N/2)],model_op1.x) + summation(obj_coef[int(N/2):N],model_op1.u), sense=minimize)

model_op1.pprint()


3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    u : Size=5, Index=u_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    x : Siz

In [4]:
opt_cbc=SolverFactory('cbc')
result=opt_cbc.solve(model_op1)

print("solver status:",result.solver.status)
print("solver termination condition:",result.solver.termination_condition)

solver status: ok
solver termination condition: optimal


Ex 1.2
Solver status is OK   
Solver termination condition is optimal.

Solver yields an optimal solution so no need of any remedy.

In [ ]:
# 1.3 printing value of objective function, decision variables and constraints
print('\nObjective = ', model_op1.objective())

print('\nDecision Variables x')
for i in col_indices_x:
  print('x[',i,'] = ', model_op1.x[i].value)

print('\nDecision Variables u')
for i in col_indices_u:
  print('u[',i,'] = ', model_op1.u[i].value)

print('\nConstraints')
model_op1.constraints.display()



Objective =  0.547955725

Decision Variables x
x[ 0 ] =  -0.046448575
x[ 1 ] =  0.18612441
x[ 2 ] =  -0.17131802
x[ 3 ] =  0.0
x[ 4 ] =  -0.14406472

Decision Variables u
u[ 0 ] =  0.046448575
u[ 1 ] =  0.18612441
u[ 2 ] =  0.17131802
u[ 3 ] =  0.0
u[ 4 ] =  0.14406472

Constraints
constraints : Size=16
    Key : Lower : Body               : Upper
      1 :   1.0 : 0.9999996250000001 :  None
      2 :   1.0 : 0.9999997599999997 :  None
      3 :   1.0 :  1.458337769999999 :  None
      4 :   1.0 : 1.0000002550000002 :  None
      5 :   1.0 :  1.000000215000001 :  None
      6 :   1.0 : 5.2549776999999995 :  None
      7 :   0.0 :         0.09289715 :  None
      8 :   0.0 :                0.0 :  None
      9 :   0.0 :         0.34263604 :  None
     10 :   0.0 :                0.0 :  None
     11 :   0.0 :         0.28812944 :  None
     12 :   0.0 :                0.0 :  None
     13 :   0.0 :         0.37224882 :  None
     14 :   0.0 :                0.0 :  None
     15 :   0.0 :  

1.3 Objective value and value of decision variables is printed above.

Constraints 1,2,4,5,8,10,12,14,15 and 16 are active.  
Remaining constraints are inactive.

On comapring $x_{i}$ with $u_{i}$ we can see that there are three $x_{i}$ which are taking negative value at the optimal and there corresponding $u_{i}$ values are of the same magnitude but positive.   
One of the remaining two $x_{i}$ is having positive value and other is 0, for these $x_{i}$ the corresponding $u_{i}$ values are same.  
From these observations it is verified that the approach we have used to solve this problem is actually giving absolute values, so we were able to convert non linear absolute value function type optimization problem into a linear program and solve it.

**EX1 (OP2)**

In [6]:
coef = np.loadtxt('lab6_practice_coef_OP2.txt', delimiter=',')

model_op2 = ConcreteModel()

N=coef.shape[1]-1
M=coef.shape[0]-1

col_indices_a=np.arange(N)
col_indices_b=np.arange(N)

row_indices=np.arange(M)

model_op2.a = Var(col_indices_a, domain=NonNegativeReals)
model_op2.b = Var(col_indices_b, domain=NonNegativeReals)

constr_coef=coef[1:,:-1]
constr_rhs = coef[1:,-1]

obj_coef = coef[0,:-1]

model_op2.constraints = ConstraintList()

for i in row_indices:
  model_op2.constraints.add(summation(constr_coef[i][0:N],model_op2.a) - summation(constr_coef[i][0:N],model_op2.b)  >= constr_rhs[i])


model_op2.objective = Objective(expr = summation(obj_coef[0:N],model_op2.a) + summation(obj_coef[0:N],model_op2.b), sense=minimize)

model_op2.pprint()





3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

2 Var Declarations
    a : Size=5, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    b : Size=5, Index=b_index
        Key : Lowe

In [7]:
opt_cbc=SolverFactory('cbc')
result=opt_cbc.solve(model_op2)

print("solver status:",result.solver.status)
print("solver termination condition:",result.solver.termination_condition)

solver status: ok
solver termination condition: optimal


Ex 1.9
Solver status is OK   
Solver termination condition is optimal.

Solver yields an optimal solution so no need of any remedy.

In [8]:
print('\nObjective = ', model_op2.objective())

print('\nDecision Variables a')
for i in col_indices_a:
  print('a[',i,'] = ', model_op2.a[i].value)

print('\nDecision Variables b')
for i in col_indices_u:
  print('b[',i,'] = ', model_op2.b[i].value)

print('\nConstraints')
model_op2.constraints.display()


Objective =  0.547955725

Decision Variables a
a[ 0 ] =  0.0
a[ 1 ] =  0.18612441
a[ 2 ] =  0.0
a[ 3 ] =  0.0
a[ 4 ] =  0.0

Decision Variables b
b[ 0 ] =  0.046448575
b[ 1 ] =  0.0
b[ 2 ] =  0.17131802
b[ 3 ] =  0.0
b[ 4 ] =  0.14406472

Constraints
constraints : Size=6
    Key : Lower : Body               : Upper
      1 :   1.0 :  0.999999625000001 :  None
      2 :   1.0 : 0.9999997599999997 :  None
      3 :   1.0 :         1.45833777 :  None
      4 :   1.0 : 1.0000002549999998 :  None
      5 :   1.0 :        1.000000215 :  None
      6 :   1.0 :  5.254977699999998 :  None


1.10 
Objective value and value of decision variables is printed above.

Constraints 1,2,4 and 5 are active.  
Remaining constraints are inactive.

In [9]:
#1.11 code to print x[i] values from a[i] and b[i]
for i in col_indices_a:
  print("x[",i,"]=",model_op2.a[i].value-model_op2.b[i].value)

x[ 0 ]= -0.046448575
x[ 1 ]= 0.18612441
x[ 2 ]= -0.17131802
x[ 3 ]= 0.0
x[ 4 ]= -0.14406472


1.12 
On comaparing the values of decision variables $x_{i}$ obtained from both the approaches that is OP1 and OP2, it is observed that they are exactly same.